# Notebook App UI Example

This notebook demonstrates the in-notebook app interface letting you interact with a langchain app inside this notebook.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import sys

from trulens.core import TruSession
from trulens.core.utils.keys import check_keys
from trulens.dashboard import run_dashboard
from trulens.dashboard.appui import AppUI

# If running from github repo, can use this:
sys.path.append(str(Path().cwd().parent.parent.resolve()))

from pprint import PrettyPrinter

pp = PrettyPrinter()


check_keys("OPENAI_API_KEY")


tru = TruSession()
tru.reset_database()  # if needed
run_dashboard(force=True, _dev=Path().cwd().parent.parent.resolve())

## langchain example

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import PromptTemplate
from langchain_community.llms import OpenAI
from trulens.core.instruments import instrument
from trulens.instrument.langchain import TruChain

llm = OpenAI(temperature=0.9, max_tokens=128)

# Conversation memory.
memory = ConversationSummaryBufferMemory(
    k=4,
    max_token_limit=64,
    llm=llm,
)

# Conversational app puts it all together.
app = ConversationChain(llm=llm, memory=memory)

instrument.method(PromptTemplate, "format")

truchain = TruChain(app, app_name="langchain_app")

In [ ]:
# Interesting Select.App queries to look at:
# - app.memory.chat_memory.messages[:].content
# - app.memory.moving_summary_buffer
# - app.prompt.template

# Interesting Select.Record queries to look at:
# - app.memory.save_context[0].args
# - app.prompt.format.args.kwargs
# - app.prompt.format.rets
# The last two need to instrument PromptTemplate as above.

aui = AppUI(
    app=truchain,
    app_selectors=[
        "app.memory.chat_memory.messages[:].content",
        "app.memory.moving_summary_buffer",
        "app.prompt.template",
    ],
    record_selectors=[
        "app.memory.save_context[0].args",
        "app.prompt.format.args.kwargs",
        "app.prompt.format.rets",
    ],
)
aui.widget

## llama_index example

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.readers.web import SimpleWebPageReader
from trulens.instrument.llamaindex import TruLlama

documents = SimpleWebPageReader(html_to_text=True).load_data([
    "http://paulgraham.com/worked.html"
])
index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine()

trullama = TruLlama(query_engine, app_name="llama_index_app")

In [ ]:
aui = AppUI(
    app=trullama,
    app_selectors=[],
    record_selectors=[
        "app.retriever.retrieve[0].rets[:].score",
        "app.retriever.retrieve[0].rets[:].node.text",
    ],
)
aui.widget

## basic app example

In [ ]:
from trulens.core import TruBasic


def custom_application(prompt: str) -> str:
    return f"a useful response to {prompt}"


trubasic = TruBasic(custom_application, app_name="basic_app")

In [ ]:
aui = AppUI(
    app=trubasic,
    app_selectors=[  # nothing interesting to display here
    ],
    record_selectors=[
        "app._call[0].args.args[:]",
    ],
)
aui.widget

## custom app example

In [ ]:
from trulens.core import TruCustomApp

from examples.dev.dummy_app.app import DummyApp  # our custom app

# Create custom app:
app = DummyApp()

# Create trulens wrapper:
trucustom = TruCustomApp(
    app=app,
    app_name="custom_app",
    # Make sure to specify using the bound method, bound to self=app.
    main_method=app.respond_to_query,
)

In [ ]:
aui = AppUI(
    app=trucustom,
    app_selectors=["app.memory.messages[:]"],
    record_selectors=[
        "app.retriever.retrieve_chunks[0].rets",
    ],
)
aui.widget